In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.models import vit_b_16  # 修改1：导入ViT-B/16模型
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, auc, matthews_corrcoef
import seaborn as sns
from tqdm import tqdm
import random

# 设置随机种子保证可重复性
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)

# 检查GPU可用性
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 自定义数据集类（保持不变）
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images = []
        
        for cls_name in self.classes:
            cls_dir = os.path.join(root_dir, cls_name)
            for img_name in os.listdir(cls_dir):
                self.images.append((os.path.join(cls_dir, img_name), self.class_to_idx[cls_name]))
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# 数据增强变换（保持不变）
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ViT默认输入尺寸
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 创建数据集和数据加载器（路径需自行确认）
train_dataset = CustomDataset(r'C:\Users\Lenovo\Desktop\dataset4\train', transform=train_transform)
val_dataset = CustomDataset(r'C:\Users\Lenovo\Desktop\dataset4\val', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=0)

# 修改模型定义部分（关键修改）
class ViT(nn.Module):
    def __init__(self, num_classes=4):
        super(ViT, self).__init__()
        # 加载预训练ViT-B/16
        self.model = vit_b_16(weights='IMAGENET1K_V1')  # 修改2：使用新权重加载方式
        # 修改分类头（ViT的分类头在heads.head）
        self.model.heads.head = nn.Linear(self.model.heads.head.in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)

# 初始化模型、损失函数和优化器（修改3：使用新模型类）
model = ViT(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00002)

# 训练和验证函数（保持不变）
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(dataloader, desc="Training"):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / len(dataloader)
    train_acc = 100 * correct / total
    
    return train_loss, train_acc

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validating"):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    val_loss = running_loss / len(dataloader)
    val_acc = 100 * correct / total
    cm = confusion_matrix(all_labels, all_preds)
    
    return val_loss, val_acc, cm

# 训练参数（保持不变）
num_epochs = 500
best_val_acc = 0.0
best_model_state = None
train_losses = []
train_accs = []
val_losses = []
val_accs = []

# 训练循环（保持不变）
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, cm = validate(model, val_loader, criterion, device)
    
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict()
        torch.save(best_model_state, 'best_model_4class.pth')
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

# 绘制训练曲线（保持不变）
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy Curve')
plt.legend()

plt.tight_layout()
plt.savefig('training_curves_4class.png')
plt.show()

# 加载最佳模型（保持不变）
model.load_state_dict(torch.load('best_model_4class.pth'))
model.eval()

# 验证集评估（保持不变）
all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        _, preds = torch.max(outputs.data, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

# 计算混淆矩阵（保持不变）
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=train_dataset.classes, 
            yticklabels=train_dataset.classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix_4class.png')
plt.show()

# 计算分类报告（保持不变）
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))

# 计算MCC（修改为多分类版本）
mcc = matthews_corrcoef(all_labels, all_preds)

# 输出结果（保持原格式）
print(f"\nBest Validation Accuracy: {best_val_acc:.2f}%")
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

# 每个类别的详细指标（保持原格式）
report = classification_report(all_labels, all_preds, target_names=train_dataset.classes, output_dict=True)
for i, class_name in enumerate(train_dataset.classes):
    cls_report = report[class_name]
    tn = cm[i, i]
    fp = cm[:, i].sum() - tn
    fn = cm[i, :].sum() - tn
    tp = tn  # 对角线元素即为TP
    
    specificity = (cm.sum() - cm[i, :].sum() - cm[:, i].sum() + tn) / (cm.sum() - cm[i, :].sum() - cm[:, i].sum() + tn + fp) if (cm.sum() - cm[i, :].sum() - cm[:, i].sum() + tn + fp) != 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
    
    print(f"\nClass: {class_name}")
    print(f"Sensitivity (Recall): {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1-score: {cls_report['f1-score']:.4f}")

Using device: cuda

Epoch 1/500


Validating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  5.58it/s]


Train Loss: 1.3410, Train Acc: 37.39%
Val Loss: 1.2975, Val Acc: 48.28%

Epoch 2/500


Validating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  6.07it/s]


Train Loss: 0.8509, Train Acc: 69.73%
Val Loss: 1.1778, Val Acc: 48.28%

Epoch 3/500


...

Validating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  6.12it/s]


Train Loss: 0.0001, Train Acc: 100.00%
Val Loss: 2.8439, Val Acc: 53.79%

Epoch 87/500


Validating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  6.25it/s]


Train Loss: 0.0001, Train Acc: 100.00%
Val Loss: 2.8519, Val Acc: 53.79%

Epoch 88/500


Validating:  11%|███████▍                                                               | 2/19 [00:00<00:04,  4.05it/s]